In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_train = pd.read_csv("/kaggle/input/llm-classification-finetuning/train.csv")
df_test = pd.read_csv("/kaggle/input/llm-classification-finetuning/test.csv")

In [ ]:
df_train.info()

In [ ]:
df_train.head()

In [ ]:
df_test.info()

In [ ]:
df_test.head()

In [ ]:
## Data Cleaning

In [ ]:
#pip install neattext

In [ ]:
#import neattext as ntx

In [ ]:
import re
import unicodedata

def clean_text(text):
    # Metni string'e dönüştür
    text = str(text)

    # 1. Küçük harfe çevir
    text = text.lower()

    # 2. Özel karakterleri kaldır (sadece harf, rakam ve boşluk kalsın)
    text = re.sub(r'[^a-z0-9ğüşöçıİĞÜŞÖÇ\s]', '', text)

    # 3. Unicode karakterleri normalize et
    text = unicodedata.normalize('NFKC', text)

    # 4. Fazla boşlukları temizle
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
text_columns = ["prompt","response_a","response_b"]

df_train[text_columns] = df_train[text_columns].applymap(clean_text)

In [ ]:
df_train.head()

In [ ]:
#one-hot-endocing

winner = ["winner_model_a","winner_model_b","winner_tie"]

df_train["result"] = df_train[winner].apply(lambda row : list(row),axis=1)

In [ ]:
df_train.head()

In [ ]:
#Soru ve cevapları birleştir
df_train["question_response"] = df_train["prompt"] + " answer: " + df_train["response_a"] + " VS " + df_train["response_b"]

In [ ]:
df = df_train[["question_response","result"]]

In [ ]:
df.sample(3)

In [ ]:
#Modeling

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
X = df["question_response"].values  # liste veya array olarak
y = np.array(df["result"].tolist())  # one-hot array

In [ ]:
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

In [ ]:
#metinleri sayısan dizine çevir
X_seq = tokenizer.texts_to_sequences(X)

In [ ]:
#maxlen'i bulalım
seq_lengths = max([len(seq) for seq in X_seq])

In [ ]:
seq_lengths

In [ ]:
#her diziyi aynı uzunluğa getirmek için başına 0 ekleyelim

X_pad = pad_sequences(X_seq,maxlen=seq_lengths, padding="pre")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import load_model

In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    LSTM(64),
    Dense(3, activation='softmax')  # 3 sınıf olduğu için
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
model.save("model.h5")

In [ ]:
model = load_model("/kaggle/input/llm-classification-model/keras/default/1/model.h5")

In [ ]:
df_test.head()

In [ ]:
text_columns = ["prompt","response_a","response_b"]

df_test[text_columns] = df_test[text_columns].applymap(clean_text)

In [ ]:
#Soru ve cevapları birleştir
df_test["question_response"] = df_test["prompt"] + " answer: " + df_test["response_a"] + " VS " + df_test["response_b"]

In [ ]:
X_ = df_test["question_response"].values  # liste veya array olarak

In [ ]:
#metinleri sayısan dizine çevir
X_seq_ = tokenizer.texts_to_sequences(X_)

In [ ]:
#her diziyi aynı uzunluğa getirmek için başına 0 ekleyelim

X_pad_ = pad_sequences(X_seq_,maxlen=seq_lengths, padding="pre")

In [ ]:
predicts = model.predict(X_pad_)

In [ ]:
predicts

In [ ]:
df_test[["winner_model_a","winner_model_b","winner_model_tie"]] = predicts

In [ ]:
df_submission = df_test[["id","winner_model_a","winner_model_b","winner_model_tie"]]

In [ ]:
df_submission.to_csv("submission.csv")